In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#export
import re
import lib
import logging
import time
import pandas as pd
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

In [4]:
# x = lib.DatabaseIO().read_data('triggers')
# x.head()

In [7]:
#export
api_token = lib.config['telegram']['api_token']

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)


class HenryBot:
    def __init__(self):
        self.db_conn = lib.DatabaseIO()
        self.triggers = self.db_conn.read_data('triggers')

    @staticmethod
    def start(update, context):
        """Start interacting with Henry!"""
        update.message.reply_text('Hi! I am HenryBot and you\'ve triggered me. You can add new triggers using /add. Get more information using /help')

    def update_triggers(self):
        self.db_conn.write_data(self.triggers, 'triggers')

    def add(self, update, context):
        """Add a new trigger"""
        try:
            value = update.message.text.split(' ', 1)[1]
            trigger, response = map(str.strip, value.split(':', 1))
            self.triggers.loc[trigger] = [response]
            update.message.reply_text(f'\'{trigger}\' added to the triggers')
            self.update_triggers()
        except IndexError:
            update.message.reply_text(f"No correctly formatted trigger found. Add a new trigger by typing /add <trigger>:<response>")

    def delete(self, update, context):
        """Delete a certain trigger from the list of triggers."""
        try:
            trigger = update.message.text.split(' ', 1)[1]
            self.triggers = self.triggers.drop(trigger)
            update.message.reply_text(f'\'{trigger}\' deleted from triggers.')
            self.update_triggers()
        except IndexError:
            update.message.reply_text(f'Please add a trigger to remove. Format is /delete <trigger>')

    def get_triggers(self, update, context):
        """Send the available triggers"""
        # respond with only the triggers cause the responses give a message size error
        update.message.reply_text('\n'.join(self.triggers.index.tolist()))

    def triggered(self, update, context):
        """Respond with the response associated with the trigger."""
        text = update.message.text
        user = update.message.from_user
        message = ''

        # Respond to 'I am'
        matches = re.search(r'ik ben (\w+)', text, re.I)
        if matches is not None:
            message += f'Hoi {matches.group(1)}, ik ben HenryBot\n'

        # Respond to added triggers
        for trigger, row in self.triggers.iterrows():
            regex = r'(\b|\A)\S{0,1}' + trigger + '\S{0,1}(\b){0,1}'
            if re.search(regex, text, re.I):
                assert isinstance(self.triggers.loc[trigger], pd.Series), f'Trigger query response is not a series. Likely duplicate trigger.'
                message += self.triggers.loc[trigger, 'response'] + '\n'

        message = message.format(user=user.first_name)
        if message:
            update.message.reply_text(message)

    @staticmethod
    def help(update, context):
        """Send a message when the command /help is issued."""
        update.message.reply_text('Help!')
        time.sleep(3)
        update.message.reply_text('Just kidding! HAHAHAHAHAHA')
        update.message.reply_text('''Add triggers by doing: /add <trigger>:<respons>
Remove triggers by doing: /delete <trigger>
Get all triggers by doing: /triggers\
        ''')


def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    """Start the bot."""
    henry_bot = HenryBot()
    updater = Updater(api_token, use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # on different commands - answer in Telegram
    dp.add_handler(CommandHandler("start", henry_bot.start))
    dp.add_handler(CommandHandler("add", henry_bot.add))
    dp.add_handler(CommandHandler("delete", henry_bot.delete))
    dp.add_handler(CommandHandler("triggers", henry_bot.get_triggers))
    dp.add_handler(CommandHandler("help", henry_bot.help))

    # on noncommand i.e message - respond appropriately to the message on Telegram
    dp.add_handler(MessageHandler(Filters.text, henry_bot.triggered))

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()
    updater.idle()


In [8]:
main()

2019-05-23 23:15:21,505 - lib - INFO - Reading data from triggers...
2019-05-23 23:15:24,289 - lib - INFO - Done.
2019-05-23 23:16:00,310 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...


In [37]:
a = ['ATTACHMENT_TYPES', 'MESSAGE_TYPES', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__metaclass__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_effective_attachment', '_id_attrs', '_parse_html', '_parse_markdown', '_quote', 'animation', 'audio', 'author_signature', 'bot', 'caption', 'caption_entities', 'caption_html', 'caption_html_urled', 'caption_markdown', 'caption_markdown_urled', 'channel_chat_created', 'chat', 'chat_id', 'connected_website', 'contact', 'date', 'de_json', 'delete', 'delete_chat_photo', 'document', 'edit_caption', 'edit_date', 'edit_media', 'edit_reply_markup', 'edit_text', 'effective_attachment', 'entities', 'forward', 'forward_date', 'forward_from', 'forward_from_chat', 'forward_from_message_id', 'forward_signature', 'from_user', 'game', 'group_chat_created', 'invoice', 'left_chat_member', 'link', 'location', 'media_group_id', 'message_id', 'migrate_from_chat_id', 'migrate_to_chat_id', 'new_chat_members', 'new_chat_photo', 'new_chat_title', 'parse_caption_entities', 'parse_caption_entity', 'parse_entities', 'parse_entity', 'passport_data', 'photo', 'pinned_message', 'reply_animation', 'reply_audio', 'reply_contact', 'reply_document', 'reply_html', 'reply_location', 'reply_markdown', 'reply_media_group', 'reply_photo', 'reply_sticker', 'reply_text', 'reply_to_message', 'reply_venue', 'reply_video', 'reply_video_note', 'reply_voice', 'sticker', 'successful_payment', 'supergroup_chat_created', 'text', 'text_html', 'text_html_urled', 'text_markdown', 'text_markdown_urled', 'to_dict', 'to_json', 'venue', 'video', 'video_note', 'voice']
{'id': 6054587, 'first_name': 'Bauke', 'is_bot': False, 'last_name': 'Brenninkmeijer', 'username': 'Bauke', 'language_code': 'en'}
a

['ATTACHMENT_TYPES',
 'MESSAGE_TYPES',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__metaclass__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_effective_attachment',
 '_id_attrs',
 '_parse_html',
 '_parse_markdown',
 '_quote',
 'animation',
 'audio',
 'author_signature',
 'bot',
 'caption',
 'caption_entities',
 'caption_html',
 'caption_html_urled',
 'caption_markdown',
 'caption_markdown_urled',
 'channel_chat_created',
 'chat',
 'chat_id',
 'connected_website',
 'contact',
 'date',
 'de_json',
 'delete',
 'delete_chat_photo',
 'document',
 'edit_caption',
 'edit_date',
 'edit_media',
 'edit_reply_markup',
 'edit_text',
 'effective_attachment',
 'entities',
 'forward',
 'forward_date',


In [ ]:
#export
if __name__ == '__main__':
    main()

In [9]:
!python notebook2script.py bot.ipynb

Converted bot.ipynb to bot.py
